In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from UliEngineering.EngineerIO import format_value
from si_prefix import si_format
# import plecs_helper as helper
%matplotlib
%matplotlib inline

# Imports and setup
from pint import UnitRegistry
from scipy.signal import find_peaks
from scipy.optimize import fsolve

# pandas display using scientific notation
# pd.set_option('display.float_format', lambda x: f'{x:.3e}')

# use pint
units = UnitRegistry()
units.default_format = "~P.2f"

Using matplotlib backend: <object object at 0x00000234B3FC0510>


C:\Users\ieykamp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Lab 11 Prep
### Ian, Lauren, Melissa

For Lab 10, we used the provided feedback components `C1 = 47nF`, `R5 = 510Ohm`, and `C2 = 10nF`, with `C2` in parallel with `(C1 + R5)` in series.

In [2]:
def find_values(Rf1_unitless):
    Vout = 10 * units.volt
    # Rf1 = 3.3 * units.kiloohm
    Rf1 = Rf1_unitless * units.ohm
    Rf2 = Rf1 / 3 # 1 * units.kiloohm
    Vout_calc = 2.5 * units.volt * (Rf1 + Rf2) / Rf2
    # print(f"Output voltage based on voltage divider: {Vout_calc}")

    wc = 50 * units.kilohertz * units.revolutions / 8
    # def get_Zc(my_w):
    #     # my_Zc = 1 / (1 / (R5 + 1 / (1j * my_w * C1)) + 1 / (1 / (1j * my_w * C2))) / 4.3e3 # kOhm
    #     my_Zc = 1 / (1 / (Rc1 + 1 / (2 * np.pi * 1j * my_w * Cc1)) + 1 / (1 / (2 * np.pi * 1j * my_w * Cc2))) # / 4.3e3 # kOhm
    #     return my_Zc
    # Zc_gc = get_Zc(wc)
    # print(f"Zc(w = wc): {np.abs(Zc_gc).to_compact(units.ohm)}")

    angle_boost = 66 * units.degree
    K = np.tan((angle_boost / 2 + 45 * units.degree))
    Kp = 1 / 3
    wz = wc / K
    wp = wc * K
    Gc0 = Kp * wc * np.sqrt((1 + K ** (-2)) / (1 + K ** 2))
    Cc1 = 1 / (Gc0 * Rf1)
    Rc1 = 1 / (wz * Cc1)
    Cc2 = 1 / (wp * Rc1)

    print(f"K: {K},\t Gc0: {Gc0.to_compact()},\t Rf1: {Rf1.to_compact(units.ohm)},\t Rf2: {Rf2.to_compact(units.ohm)},\t Rc1: {Rc1.to_compact(units.ohm)},\t Cc1: {Cc1.to_compact(units.farad)},  \t Cc2: {Cc2.to_compact(units.farad)}")

for R in [2, 2.2, 2.7, 3.3, 3.9, 4.3, 4.7, 5.1, 5.6, 6.2, 6.8, 7.5, 8.2, 10]:
    if R == 7.5:
        print()
    find_values(R * 1e3)
    if R == 7.5:
        print()

K: 4.70,	 Gc0: 442.83 Hz·turn,	 Rf1: 2.00 kΩ,	 Rf2: 666.67 Ω,	 Rc1: 666.67 Ω,	 Cc1: 179.70 nF,  	 Cc2: 8.12 nF
K: 4.70,	 Gc0: 442.83 Hz·turn,	 Rf1: 2.20 kΩ,	 Rf2: 733.33 Ω,	 Rc1: 733.33 Ω,	 Cc1: 163.37 nF,  	 Cc2: 7.38 nF
K: 4.70,	 Gc0: 442.83 Hz·turn,	 Rf1: 2.70 kΩ,	 Rf2: 900.00 Ω,	 Rc1: 900.00 Ω,	 Cc1: 133.11 nF,  	 Cc2: 6.01 nF
K: 4.70,	 Gc0: 442.83 Hz·turn,	 Rf1: 3.30 kΩ,	 Rf2: 1.10 kΩ,	 Rc1: 1.10 kΩ,	 Cc1: 108.91 nF,  	 Cc2: 4.92 nF
K: 4.70,	 Gc0: 442.83 Hz·turn,	 Rf1: 3.90 kΩ,	 Rf2: 1.30 kΩ,	 Rc1: 1.30 kΩ,	 Cc1: 92.16 nF,  	 Cc2: 4.16 nF
K: 4.70,	 Gc0: 442.83 Hz·turn,	 Rf1: 4.30 kΩ,	 Rf2: 1.43 kΩ,	 Rc1: 1.43 kΩ,	 Cc1: 83.58 nF,  	 Cc2: 3.78 nF
K: 4.70,	 Gc0: 442.83 Hz·turn,	 Rf1: 4.70 kΩ,	 Rf2: 1.57 kΩ,	 Rc1: 1.57 kΩ,	 Cc1: 76.47 nF,  	 Cc2: 3.45 nF
K: 4.70,	 Gc0: 442.83 Hz·turn,	 Rf1: 5.10 kΩ,	 Rf2: 1.70 kΩ,	 Rc1: 1.70 kΩ,	 Cc1: 70.47 nF,  	 Cc2: 3.18 nF
K: 4.70,	 Gc0: 442.83 Hz·turn,	 Rf1: 5.60 kΩ,	 Rf2: 1.87 kΩ,	 Rc1: 1.87 kΩ,	 Cc1: 64.18 nF,  	 Cc2: 2.90 nF
K: 4.70,	 Gc0: 442

In [3]:
# x = 90 * units.degrees
# print(x)
# y = x.to(units.radians)
# print(y)

# print(x * units.meters)
# print(y * units.meters)

# print((x * units.meters).to(units.meters))
# print((y * units.meters).to(units.meters))

a = 10 * units.hertz
print(a)
b = a.to(units.radian / units.second)
print(b)


10.00 Hz
10.00 rad/s
